## Indian Premier League (IPL) Regression Analysis

In [1]:
%%capture
# Due to the configuration of the base Jupter image, the following imports are required for the regressions in the assignment to report the correct metrics

import sys 
!{sys.executable} -m pip uninstall statsmodels --yes 
!{sys.executable} -m pip uninstall numpy --yes
!{sys.executable} -m pip uninstall pandas --yes 
!{sys.executable} -m pip uninstall patsy --yes 
!{sys.executable} -m pip install numpy==1.17
!{sys.executable} -m pip install pandas==1.0
!{sys.executable} -m pip install patsy==0.5.2
!{sys.executable} -m pip install statsmodels==0.11.1

In [2]:
# As usual, we begin by loading the packages we will need

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [13]:
# Now we load the data

IPL=pd.read_excel("Assignment Data/Week 5/IPL (assignment) data.xlsx")
IPL.columns

Index(['year', 'team', 'played', 'won', 'lost', 'noresult', 'points',
       'netrunrate', 'champions', 'runnersup', 'third', 'fourth', 'salaries'],
      dtype='object')

In [15]:
#Over the next few commands we create the relative salary(team's salary/sum of salaries)
sumsal=IPL.groupby(['year'])['salaries'].sum().reset_index().rename(columns={'salaries':'allsal'})
sumsal

,year,allsal
0,2008,34105000
1,2009,33445000
2,2010,33075000
3,2011,62210000
4,2012,59706250
5,2013,81535000
6,2014,73973332
7,2015,65242665
8,2016,63483666
9,2017,63014833


In [16]:
IPL=pd.merge(IPL, sumsal, on=["year"], how="left")
IPL

,year,team,played,won,lost,noresult,points,netrunrate,champions,runnersup,third,fourth,salaries,allsal
0,2008,Chennai Super Kings,14,8,6,0,16,-0.192,0,1,0,0,5825000,34105000
1,2009,Chennai Super Kings,14,8,5,1,17,0.951,0,0,0,0,6765000,33445000
2,2010,Chennai Super Kings,14,7,7,0,14,0.274,1,0,0,0,4890000,33075000
3,2011,Chennai Super Kings,14,9,5,0,18,0.443,1,0,0,0,6330000,62210000
4,2012,Chennai Super Kings,16,8,7,1,17,0.100,0,1,0,0,7900000,59706250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,2014,Sunrisers Hyderabad,14,6,8,0,12,-0.399,0,0,0,0,9803333,73973332
88,2015,Sunrisers Hyderabad,14,7,7,0,14,-0.239,0,0,0,0,8104667,65242665
89,2016,Sunrisers Hyderabad,14,8,6,0,16,0.245,1,0,0,0,8747666,63483666
90,2017,Sunrisers Hyderabad,14,8,5,1,17,0.599,0,0,0,1,8200500,63014833


In [18]:
IPL['relsal']=IPL['salaries']/IPL['allsal']
IPL

,year,team,played,won,lost,noresult,points,netrunrate,champions,runnersup,third,fourth,salaries,allsal,relsal
0,2008,Chennai Super Kings,14,8,6,0,16,-0.192,0,1,0,0,5825000,34105000,0.170796
1,2009,Chennai Super Kings,14,8,5,1,17,0.951,0,0,0,0,6765000,33445000,0.202272
2,2010,Chennai Super Kings,14,7,7,0,14,0.274,1,0,0,0,4890000,33075000,0.147846
3,2011,Chennai Super Kings,14,9,5,0,18,0.443,1,0,0,0,6330000,62210000,0.101752
4,2012,Chennai Super Kings,16,8,7,1,17,0.100,0,1,0,0,7900000,59706250,0.132314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,2014,Sunrisers Hyderabad,14,6,8,0,12,-0.399,0,0,0,0,9803333,73973332,0.132525
88,2015,Sunrisers Hyderabad,14,7,7,0,14,-0.239,0,0,0,0,8104667,65242665,0.124223
89,2016,Sunrisers Hyderabad,14,8,6,0,16,0.245,1,0,0,0,8747666,63483666,0.137794
90,2017,Sunrisers Hyderabad,14,8,5,1,17,0.599,0,0,0,1,8200500,63014833,0.130136


In [23]:
#we find wpc and last season's wpc
IPL['wpc']=IPL['won']/(IPL['played']-IPL['noresult'])
IPL.sort_values(by=['team','year'], ascending=True)
IPL['lagwpc']=IPL.groupby('team')['wpc'].shift(1)
IPL

,year,team,played,won,lost,noresult,points,netrunrate,champions,runnersup,third,fourth,salaries,allsal,relsal,wpc,lagwpc
0,2008,Chennai Super Kings,14,8,6,0,16,-0.192,0,1,0,0,5825000,34105000,0.170796,0.571429,NaN
1,2009,Chennai Super Kings,14,8,5,1,17,0.951,0,0,0,0,6765000,33445000,0.202272,0.615385,0.571429
2,2010,Chennai Super Kings,14,7,7,0,14,0.274,1,0,0,0,4890000,33075000,0.147846,0.500000,0.615385
3,2011,Chennai Super Kings,14,9,5,0,18,0.443,1,0,0,0,6330000,62210000,0.101752,0.642857,0.500000
4,2012,Chennai Super Kings,16,8,7,1,17,0.100,0,1,0,0,7900000,59706250,0.132314,0.533333,0.642857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,2014,Sunrisers Hyderabad,14,6,8,0,12,-0.399,0,0,0,0,9803333,73973332,0.132525,0.428571,0.625000
88,2015,Sunrisers Hyderabad,14,7,7,0,14,-0.239,0,0,0,0,8104667,65242665,0.124223,0.500000,0.428571
89,2016,Sunrisers Hyderabad,14,8,6,0,16,0.245,1,0,0,0,8747666,63483666,0.137794,0.571429,0.500000
90,2017,Sunrisers Hyderabad,14,8,5,1,17,0.599,0,0,0,1,8200500,63014833,0.130136,0.615385,0.571429


In [24]:
#we create the regression models that were asked in the assignment
reg1=smf.ols(formula= 'wpc ~ relsal', data=IPL).fit()
print(reg1.summary())

                            OLS Regression Results                            
Dep. Variable:                    wpc   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                    0.1622
Date:                Wed, 01 Jun 2022   Prob (F-statistic):              0.688
Time:                        11:10:01   Log-Likelihood:                 45.259
No. Observations:                  92   AIC:                            -86.52
Df Residuals:                      90   BIC:                            -81.47
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4752      0.063      7.563      0.0

In [25]:
reg2=smf.ols(formula= 'wpc ~ relsal+lagwpc', data=IPL).fit()
print(reg2.summary())

                            OLS Regression Results                            
Dep. Variable:                    wpc   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     1.591
Date:                Wed, 01 Jun 2022   Prob (F-statistic):              0.210
Time:                        11:10:45   Log-Likelihood:                 43.314
No. Observations:                  79   AIC:                            -80.63
Df Residuals:                      76   BIC:                            -73.52
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3489      0.087      4.003      0.0

In [26]:
reg3=smf.ols(formula= 'wpc ~ relsal+lagwpc+C(team)', data=IPL).fit()
print(reg3.summary())

                            OLS Regression Results                            
Dep. Variable:                    wpc   R-squared:                       0.297
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     2.117
Date:                Wed, 01 Jun 2022   Prob (F-statistic):             0.0245
Time:                        11:11:28   Log-Likelihood:                 55.639
No. Observations:                  79   AIC:                            -83.28
Df Residuals:                      65   BIC:                            -50.10
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------